### Data Transformation

In [11]:
1+1

2

In [12]:
import os
%pwd

'/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer'

In [13]:
os.chdir("../")

In [14]:
%pwd

'/Users/brunoferreira/Downloads/End-to-End MLOps'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [16]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [17]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config



In [18]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

### Data Transformation Component


In [19]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))



    

In [23]:
def _convert_examples_to_features(self, example_batch):
    input_encodings = self.tokenizer(
        example_batch["dialogue"],
        max_length=1024,
        truncation=True
    )

    target_encodings = self.tokenizer(
        text_target=example_batch["summary"],
        max_length=128,
        truncation=True
    )

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"],
    }

DataTransformation.convert_examples_to_features = _convert_examples_to_features
print("hotfix applied")


hotfix applied


In [ ]:
from pathlib import Path

project_root = Path("/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer")

config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()

# force path
data_transformation_config.data_path = project_root / "artifacts" / "data_ingestion" / "samsum_dataset"
data_transformation_config.root_dir = project_root / "artifacts" / "data_transformation"

print(data_transformation_config.data_path)
print(data_transformation_config.data_path.exists())  # as to be truth

data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2026-02-22 15:51:09,716: INFO: common: yaml file: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/config/config.yaml loaded successfully]
[2026-02-22 15:51:09,722: INFO: common: yaml file: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/params.yaml loaded successfully]
[2026-02-22 15:51:09,723: INFO: common: created directory at: artifacts]
[2026-02-22 15:51:09,723: INFO: common: created directory at: artifacts/data_transformation]
/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/artifacts/data_ingestion/samsum_dataset
True
[2026-02-22 15:51:09,886: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-22 15:51:09,900: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json "HTTP/1.1 200 OK"]
[2026-02-22 15:51:10,025: INFO: _client: HTT

Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 157664.66 examples/s]
